<a href="https://colab.research.google.com/github/KauaHenSilva/python_nn_iris_database/blob/main/iris_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recuperar arquivos

In [1]:
import os
tipo_de_recuperacao = "Ambos" #@param ["Download", "Drive", "Ambos"]
def repostirio_para_download():
  if os.path.exists('python_download_file_drive'):
    !cd python_download_file_drive && git pull
  else:
    !git clone https://github.com/KauaHenSilva/python_download_file_drive
  %pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

## Recuperação por download

In [2]:
if tipo_de_recuperacao in ["Download", "Ambos"]:
  from google.colab import auth
  from googleapiclient.discovery import build
  auth.authenticate_user()
  repostirio_para_download()
  !python "python_download_file_drive/main.py" "https://drive.google.com/drive/folders/1j_ObqV-_FGYoy20FARnyrSAbA4kKp0qj?usp=sharing" arquivos

Already up to date.
Baixando arquivos: 100% 1/1 [00:00<00:00, 17403.75arquivo/s]


## Recuperação por drive

In [3]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
# @markdown > https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/python_nn_iris_database" #@param {type:"string"}

In [4]:
if tipo_de_recuperacao in ["Drive", "Ambos"]:
  from google.colab import drive
  drive.mount('/content/drive')
  if not os.path.exists(path_atalho_drive):
    raise Exception("O caminho especificado não existe no drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Referenciar arquivo no drive ou download

In [5]:
def get_referencia_valida(path):
    path_download = os.path.join("/content/arquivos", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# Iris

In [6]:
!pip install tensorflow==2.16.1 scikeras==0.13.0  scikit-learn==1.5.0 --quiet

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import scikeras

In [8]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.5.0', '0.13.0')

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend as k
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier

In [10]:
path_base = get_referencia_valida('iris.csv')

In [11]:
base = pd.read_csv(path_base)
base

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [12]:
x = base.iloc[:, 0:4].values
y = base.iloc[:, 4].values

In [13]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = np_utils.to_categorical(y)

In [14]:
def get_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(4,)),
      tf.keras.layers.Dense(units=4, activation='relu'),
      tf.keras.layers.Dense(units=4, activation='relu'),
      tf.keras.layers.Dense(units=3, activation='softmax')
  ])
  rede_neural.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  return rede_neural

In [15]:
rede_neural = KerasClassifier(model=get_rede, epochs=250, batch_size=10)

In [16]:
resultados = cross_val_score(estimator=rede_neural, X = x, y = y, cv = 10, scoring = 'accuracy')

Epoch 1/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - categorical_accuracy: 0.3740 - loss: 1.1942
Epoch 2/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.3433 - loss: 1.2082 
Epoch 3/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.3651 - loss: 1.1412 
Epoch 4/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3448 - loss: 1.1434 
Epoch 5/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.3205 - loss: 1.1510 
Epoch 6/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.3811 - loss: 1.0950 
Epoch 7/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.3633 - loss: 1.1046 
Epoch 8/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3565 - loss: 1.0970 
Epoch 9/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.4134 - loss: 1.0675 
Epoch 10/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3657 - loss: 1.0836 
Epoch 11/250
14/14 ━

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Epoch 1/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - categorical_accuracy: 0.3861 - loss: 2.0132
Epoch 2/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3934 - loss: 1.6409 
Epoch 3/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3372 - loss: 1.6910 
Epoch 4/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3893 - loss: 1.3632 
Epoch 5/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3974 - loss: 1.2315 
Epoch 6/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3468 - loss: 1.2723 
Epoch 7/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3775 - loss: 1.1593 
Epoch 8/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.3554 - loss: 1.1436 
Epoch 9/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.4079 - loss: 1.1090 
Epoch 10/250
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - categorical_accuracy: 0.441

In [17]:
resultados

array([1.        , 1.        , 1.        , 1.        , 0.93333333,
       0.93333333, 1.        , 0.        , 0.8       , 1.        ])

In [18]:
resultados.mean()

0.8666666666666668

In [19]:
resultados.std()

0.29514591494904874